In [1]:
db_id = ''
db_pw = ''
db_nm = ''

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import requests
import time
from tqdm import tqdm

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


# DB관련 함수정의
def db_connection(host_name=host_name):
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8', cursorclass=pymysql.cursors.DictCursor) # cursorclass=pymysql.cursors.DictCursor 추가     # db=db,
    return conn


def engine_upload(upload_df: object, host_name: object = 'ds', db_name: object = db_nm, table_name: object = 'ranking') -> object:
    host_url = host_url
    user_nm = db_id
    passwd = db_pw
    port_num = port_num
    db_name = db_nm
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    engine_conn = engine.connect()
    upload_df.to_sql(table_name, engine_conn, if_exists='append', index=None)
    engine_conn.close()
    engine.dispose()

In [3]:
request_headers = {
    'authority': 'kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com',
    'method': 'POST',
    'path': '/graphql',
    'scheme': 'https',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko; q=0.9,en-US; q=0.8,en; q=0.7',
    'Content-Length': '1680',
    'Content-Type': 'application/json',
    'Origin': 'https://www.goodreads.com',
    'Referer': 'https://www.goodreads.com/',
    'Sec-Ch-Ua': '"Google Chrome"; v="119", "Chromium"; v="119", "Not?A_Brand"; v="24"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "macOS",
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'X-Api-Key': 'da2-oqbnu4fbwrbtlf5xiuslyifrti',
    'Address': '99.84.133.26:443'
}

In [4]:
conn = db_connection(host_name)
book_href = pd.read_sql(f"select * from goodreads_book_href where status = 'fin';", conn)
conn.close()
print(len(book_href))
book_href.head()

2969


,id,genre,category_list,category_rank,title,utm_code,book_href,status,regist_date
0,1,art,Best Book Cover Art,1,"Hush, Hush (Hush, Hush, #1)",6339664,https://www.goodreads.com/book/show/6339664-hu...,fin,NaT
1,2,art,Best Book Cover Art,2,"Twilight (The Twilight Saga, #1)",41865,https://www.goodreads.com/book/show/41865.Twil...,fin,NaT
2,3,art,Best Book Cover Art,3,The Thirteenth Tale,40440,https://www.goodreads.com/book/show/40440.The_...,fin,NaT
3,4,art,Best Book Cover Art,4,"New Moon (The Twilight Saga, #2)",49041,https://www.goodreads.com/book/show/49041.New_...,fin,NaT
4,5,art,Best Book Cover Art,5,"Fallen (Fallen, #1)",6487308,https://www.goodreads.com/book/show/6487308-fa...,fin,NaT


In [5]:
book_href.drop_duplicates(['utm_code', 'book_href'], ignore_index=True, inplace=True)
print(len(book_href))

2969


In [6]:
conn = db_connection(host_name)
book_detail = pd.read_sql(f"select * from goodreads_book_detail where status is null;", conn)
conn.close()
print(len(book_detail))
book_detail.head()

2966


,id,genre,genre2,category_list,category_rank,title,original_title,utm_code,isbn13,isbn10,...,price,language,description,character,awards,cover,setting,image,regist_date,status
0,2,art,"['Fantasy', 'Young Adult', 'Romance', 'Fiction...",Best Book Cover Art,2,"Twilight (The Twilight Saga, #1)",Twilight,41865,9780316015844,0316015849,...,10.99,English,About three things I was absolutely positive. ...,"['Edward Cullen', 'Jacob Black', 'Laurent', 'R...","['Georgia Peach Book Award (2007)', 'Buxtehude...",Paperback,"['Forks', 'Washington (United States)', 'Phoen...",https://images-na.ssl-images-amazon.com/images...,2023-11-17 07:33:10,None
1,3,art,"['Fiction', 'Mystery', 'Historical Fiction', '...",Best Book Cover Art,3,The Thirteenth Tale,The Thirteenth Tale,40440,9780743298025,0743298020,...,13.99,English,All children mythologize their birth...So begi...,"['Margaret Lea', 'Emmeline March', 'Adeline Ma...","['Dilys Award Nominee (2007)', 'ALA Alex Award...",Hardcover,['Angelfield (United Kingdom)'],https://images-na.ssl-images-amazon.com/images...,2023-11-17 07:33:10,None
2,4,art,"['Fantasy', 'Young Adult', 'Romance', 'Vampire...",Best Book Cover Art,4,"New Moon (The Twilight Saga, #2)",New Moon,49041,9780743298025,0743298020,...,10.99,English,There is an alternate cover edition for ISBN13...,"['Edward Cullen', 'Jacob Black', 'Bella Swan',...",['Books I Loved Best Yearly (BILBY) Awards for...,Hardcover,"['Forks', 'Washington (United States)', 'La Pu...",https://images-na.ssl-images-amazon.com/images...,2023-11-17 07:33:10,None
3,5,art,"['Young Adult', 'Fantasy', 'Romance', 'Paranor...",Best Book Cover Art,5,"Fallen (Fallen, #1)",Fallen,6487308,9780385738934,0385738935,...,8.99,English,There’s something achingly familiar about Dani...,"['Lucinda Price', 'Daniel Grigori', 'Cameron ""...",,Hardcover,"['Savannah', 'Georgia (United States)']",https://images-na.ssl-images-amazon.com/images...,2023-11-17 07:33:10,None
4,6,art,"['Young Adult', 'Fantasy', 'Romance', 'Paranor...",Best Book Cover Art,6,"Shiver (The Wolves of Mercy Falls, #1)",Shiver,6068551,9780545123266,0545123267,...,7.99,English,"For years, Grace has watched the wolves in the...","['Grace Brisbane', 'Sam Roth', 'Isabel Culpepe...","['Georgia Peach Book Award (2010)', ""Pennsylva...",Hardcover,"['Mercy Falls', 'Minnesota (United States', '2...",https://images-na.ssl-images-amazon.com/images...,2023-11-17 07:33:10,None


In [7]:
book_info = pd.merge(book_detail[['utm_code', 'isbn13', 'isbn10']],
                     book_href[['utm_code', 'title', 'book_href']], on='utm_code')
book_info.head()

,utm_code,isbn13,isbn10,title,book_href
0,41865,9780316015844,0316015849,"Twilight (The Twilight Saga, #1)",https://www.goodreads.com/book/show/41865.Twil...
1,40440,9780743298025,0743298020,The Thirteenth Tale,https://www.goodreads.com/book/show/40440.The_...
2,49041,9780743298025,0743298020,"New Moon (The Twilight Saga, #2)",https://www.goodreads.com/book/show/49041.New_...
3,6487308,9780385738934,0385738935,"Fallen (Fallen, #1)",https://www.goodreads.com/book/show/6487308-fa...
4,6068551,9780545123266,0545123267,"Shiver (The Wolves of Mercy Falls, #1)",https://www.goodreads.com/book/show/6068551-sh...


In [8]:
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
for book_num in range(0, len(book_info[:100])):
    drv.get(book_info['book_href'][book_num])
    drv.implicitly_wait(20)

    # login popup click
    try:
        drv.find_elements(By.CLASS_NAME, 'modal__close')[-1].click()
    except:
        pass

    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    try:
        review_href_get = 'https://www.goodreads.com'+soup.find('a', {'class', 'Button--transparent'})['href']
        resourceId = review_href_get.split('{"workId":')[-1].split(',"after":')[0].replace('"', '')
        time.sleep(1)

        drv.get(review_href_get)
        drv.implicitly_wait(20)

        # login popup
        try:
            drv.find_elements(By.CLASS_NAME, 'modal__close')[-1].click()
        except:
            pass

        review_df = pd.DataFrame(columns = ['title', 'isbn13', 'isbn10', 'text', 'text_cnt', 'text_like_cnt', 
                                            'text_id', 'utm_code', 'user_id', 'tags', 'rating'])

        api_url = 'https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql'

        null = None
        payload_1 = {"operationName":"getReviews",
                     "variables":
                         {"filters":{
                             "resourceType":"WORK",
                             "resourceId":resourceId,
                             "sort":"DEFAULT",
                             "languageCode":null
                             },
                          "pagination":{"limit":30}
                         },
                     "query":"query getReviews($filters: BookReviewsFilterInput!, $pagination: PaginationInput) {\n  getReviews(filters: $filters, pagination: $pagination) {\n    ...BookReviewsFragment\n    __typename\n  }\n}\n\nfragment BookReviewsFragment on BookReviewsConnection {\n  totalCount\n  edges {\n    node {\n      ...ReviewCardFragment\n      __typename\n    }\n    __typename\n  }\n  pageInfo {\n    prevPageToken\n    nextPageToken\n    __typename\n  }\n  __typename\n}\n\nfragment ReviewCardFragment on Review {\n  __typename\n  id\n  creator {\n    ...ReviewerProfileFragment\n    __typename\n  }\n  recommendFor\n  updatedAt\n  createdAt\n  spoilerStatus\n  lastRevisionAt\n  text\n  rating\n  shelving {\n    shelf {\n      name\n      webUrl\n      __typename\n    }\n    taggings {\n      tag {\n        name\n        webUrl\n        __typename\n      }\n      __typename\n    }\n    webUrl\n    __typename\n  }\n  likeCount\n  viewerHasLiked\n  commentCount\n}\n\nfragment ReviewerProfileFragment on User {\n  id: legacyId\n  imageUrlSquare\n  isAuthor\n  ...SocialUserFragment\n  textReviewsCount\n  viewerRelationshipStatus {\n    isBlockedByViewer\n    __typename\n  }\n  name\n  webUrl\n  contributor {\n    id\n    works {\n      totalCount\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment SocialUserFragment on User {\n  viewerRelationshipStatus {\n    isFollowing\n    isFriend\n    __typename\n  }\n  followersCount\n  __typename\n}\n"
                    }

        res = requests.post(api_url, headers=request_headers, data=json.dumps(payload_1))
        get_res = res.json()
        error_cnt = 0

        total_cnt = int(get_res['data']['getReviews']['totalCount'])

        for total in tqdm(range(total_cnt//30+1)):
            user_review_data = get_res['data']['getReviews']['edges']
            for i in range(len(user_review_data)):
                try:
                    text_id = user_review_data[i]['node']['id']
                    user_id = user_review_data[i]['node']['creator']['id']

                    review_text = user_review_data[i]['node']['text']

                    rating, text_like_cnt, review_tags = 0, 0, []
                    try:
                        rating = user_review_data[i]['node']['rating']
                    except:
                        pass

                    try:
                        text_like_cnt = user_review_data[i]['node']['likeCount']
                    except:
                        pass

                    try:
                        tag_list = user_review_data[i]['node']['shelving']['taggings']
                        for tag_json in tag_list:
                            review_tags.append(tag_json['tag']['name'])
                    except:
                        pass

                    data = pd.DataFrame({'title':book_info['title'][book_num], 'isbn13':book_info['isbn13'][book_num], 
                                         'isbn10':book_info['isbn10'][book_num], 'text':review_text, 'text_cnt':total_cnt,
                                         'text_like_cnt':text_like_cnt, 'text_id':text_id, 'utm_code':book_info['utm_code'][book_num], 
                                         'user_id':user_id, 'tags':[review_tags], 'rating':rating})
                    review_df = pd.concat([review_df, data], ignore_index=True)
                except:
                    error_cnt += 1

            # payload_1에 다음 순서 리뷰 수집에 필요한 토큰 추가
            payload_1['variables']['pagination']['after'] = get_res['data']['getReviews']['pageInfo']['nextPageToken']
            time.sleep(2)
            # next review
            res = requests.post(api_url, headers=request_headers, data=json.dumps(payload_1))
            get_res = res.json()
            time.sleep(2)
        
        # 텍스트 전처리
        pattern_1 = '<(/*)(\w*)(\w*)(\s*)(/*)>'
        review_df['text'] = [re.sub(pattern_1, '', review_df['text'][i]) for i in range(len(review_df['text']))]

        for i in range(len(review_df)):
            soup = BeautifulSoup(review_df['text'][i])
            review_df['text'][i] = soup.text.replace('\n', '').replace("\'", '"').replace('"', "'").replace('\xa0', '').strip()

        review_df['tags'] = review_df['tags'].apply(lambda x : str(x))

        # insert
        user_nm = db_id
        passwd = db_pw
        host_url = host_url
        port_num = port_num
        db_name = db_nm
        engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
        engine_conn = engine.connect()

        review_df.drop_duplicates(['text', 'text_id'], ignore_index=True, inplace=True)
        review_df.to_sql(f'goodreads_book_review', con=engine, if_exists='append', index=None)
        engine_conn.close()
        engine.dispose()

        # 수집한 category의 status 변경
        conn = db_connection(host_name)
        cur = conn.cursor()

        cur.execute(
            f'update goodreads_book_detail set status = "fin" where utm_code = "{book_info["utm_code"][book_num]}"'
        )
        conn.commit()

        cur.close()
        conn.close()
        
        print(f'{book_info["utm_code"][book_num]} finished~!')
        print(f'total review : {len(review_df)} / error_cnt : {error_cnt}')
        time.sleep(1)
    except:
        pass

100%|█████████████████████████████████████████| 154/154 [14:48<00:00,  5.77s/it]


4932435 finished~!
total review : 4599


 76%|███████████████████████████▉         | 1212/1605 [1:57:06<37:18,  5.70s/it]